In [199]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelBinarizer

### Step 1:	
Read in the three csv files as data frames. Combine all three data frames using admission_id into one named all_df, retaining all rows from Admissions.csv.

In [200]:
admin_df = pd.read_csv('C:/Users/rparg/Documents/Coding_Practice/Assignments/Admissions.csv').set_index('admission_id')
lab_df = pd.read_csv('C:/Users/rparg/Documents/Coding_Practice/Assignments/Lab.csv').set_index('admission_id')
tran_df = pd.read_csv('C:/Users/rparg/Documents/Coding_Practice/Assignments/Transfusions.csv').set_index('admission_id')

In [201]:
admin_df.at[8968,'discharge_date'] = '09/28/2008'
admin_df.at[6023,'discharge_date'] = '07/08/2001'
admin_df.at[7801,'discharge_date'] = '05/13/2010'
admin_df.at[7801,'admission_date'] = '2010-05-11'
admin_df.at[5643,'discharge_date'] = '11/14/2001'

In [202]:
len_stay = pd.DataFrame()
len_stay['len_stay'] = (((pd.to_datetime(admin_df['discharge_date'], format = '%m/%d/%Y')) - (pd.to_datetime(admin_df['admission_date'], format = '%Y-%m-%d'))).dt.days)

In [37]:
admin_df

,patient_id,admission_date,admission_time,discharge_date,discharge_time,hospital,age,sex,charlson_comorbidity_index,lap_score
admission_id,,,,,,,,,,
3602,34208,2011-09-07,NaN,09/11/2011,13:16,St. Joseph's Health Centre,NaN,M,1,45
2790,90930,2008-07-22,06:19,07/24/2008,17:54,St. Michael's Hospital,48.0,M,1,46
8755,97765,2002-08-24,14:28,08/27/2002,08:14,Mount Sinai Hospital,59.0,F,0,35
9260,17941,2007-01-12,09:39,01/26/2007,11:10,Sunnybrook Health Sciences Centre,57.0,M,1,18
5317,36532,2003-09-24,13:37,10/12/2003,16:23,Toronto Western Hospital,51.0,F,1,21
...,...,...,...,...,...,...,...,...,...,...
6028,86717,2002-12-27,16:49,01/11/2003,22:45,Toronto Western Hospital,NaN,F,2+,22
2408,42427,2008-01-26,12:58,01/28/2008,01:45,St. Michael's Hospital,50.0,M,1,38
6356,22610,2007-05-16,15:10,05/20/2007,07:04,Sunnybrook Health Sciences Centre,52.0,M,1,23


In [ ]:
# add in source column?
# joins - check that each one performs properly (is one-to-one)
# no duplicates
# lengths all add up correctly (dimensions)

In [243]:
ad_df = admin_df.drop(['patient_id','admission_time','admission_date','discharge_date','discharge_time'],axis = 1)
ad_df = ad_df.join(len_stay, how = 'left')
ad_df['charlson_comorbidity_index'].replace({'0':0,'1':1,'2+':2}, inplace = True)
ad_df['sex'].replace({'M':1, 'F': 2}, inplace = True)
ad_nn_df = ad_df[(ad_df['age'].notnull()) & (ad_df['charlson_comorbidity_index'].notnull())]
ad_in_df = ad_df[(ad_df['age'].isnull())].append(ad_df[(ad_df['charlson_comorbidity_index'].isnull())], sort = True).drop_duplicates()
mean_age = int(ad_df['age'].mean())
ad_in_df['age'].fillna(mean_age, inplace = True)
ad_in_df['charlson_comorbidity_index'].fillna(1, inplace = True)

In [244]:
X_train = ad_nn_df[['age','sex', 'charlson_comorbidity_index','lap_score','hospital']].append(ad_in_df[['age','sex', 'charlson_comorbidity_index','lap_score','hospital']])
X_train = X_train.reset_index()
ohe = LabelBinarizer()
ohe.fit(X_train['hospital'])
transformed = ohe.transform(X_train['hospital'])
ohe_df = pd.DataFrame(transformed, columns = ['Michael','Sinai','Sunnybrook','Western','Joseph'])
X_train = X_train.join(ohe_df, how = 'left').drop('hospital', axis = 1).set_index('admission_id').sort_index(ascending = True)

In [245]:
y_train = pd.DataFrame()
y_train['len_stay'] = ad_nn_df['len_stay'].append(ad_in_df['len_stay'])
def condition(x):
    if x<4:
        return 1
    elif x>=4 and x<9:
        return 2
    elif x>=9 and x<14:
        return 3
    elif x>=14 and x<18:
        return 4
    else:
        return 5
    
y_train = y_train['len_stay'].apply(condition)

In [246]:
X_test = ad_in_df[['age','hospital', 'charlson_comorbidity_index','lap_score','sex']]
y_test = ad_in_df['len_stay']
X_test = X_test.reset_index()
ohe = LabelBinarizer()
ohe.fit(X_test['hospital'])
transformed = ohe.transform(X_test['hospital'])
ohe_df = pd.DataFrame(transformed, columns = ['Michael','Sinai','Sunnybrook','Western','Joseph'])
X_test = X_test.join(ohe_df, how = 'left').drop('hospital', axis = 1).set_index('admission_id').sort_index(ascending = True)

In [247]:
y_test = pd.DataFrame()
y_test['len_stay'] = ad_in_df['len_stay']
def condition(x):
    if x<4:
        return 1
    elif x>=4 and x<9:
        return 2
    elif x>=9 and x<14:
        return 3
    elif x>=14 and x<18:
        return 4
    else:
        return 5
y_test = ad_in_df['len_stay'].apply(condition)

In [248]:
clf = RandomForestClassifier(max_depth=1000, random_state=12345)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=1000, random_state=12345)

In [249]:
clf.score(X_test, y_test.to_numpy().reshape(-1,1))

0.2023529411764706

In [250]:
def proximityMatrix(model, X):      

    terminals = model.apply(X)
    nTrees = terminals.shape[1]

    a = terminals[:,0]
    proxMat = 1*np.equal.outer(a, a)

    for i in range(1, nTrees):
        a = terminals[:,i]
        proxMat += 1*np.equal.outer(a, a)
    
    proxMat = proxMat / nTrees

    return proxMat
model = RandomForestClassifier(n_estimators=500, max_features=9, min_samples_leaf=10)
model.fit(X_train, y_train)
X = proximityMatrix(model, X_train)

In [251]:
nn_index = np.setdiff1d(X_train.index.to_numpy(),X_test.index.to_numpy())
in_pm_df = pd.DataFrame(X, index = X_train.index, columns = X_train.index.to_numpy()).drop(labels = nn_index).drop(labels = X_test.index, axis = 1)
in_pm_df = Z.div(Z.sum(axis=1), axis=0)

In [226]:
ad_nn_df['charlson_comorbidity_index']

count     4376
unique       3
top          1
freq      1592
Name: charlson_comorbidity_index, dtype: object

In [252]:
an_df = ((ad_nn_df['age'].sort_index())*in_pm_df).sum(axis = 1).astype('int')
cn_df = ((ad_nn_df['charlson_comorbidity_index'].sort_index())*in_pm_df).sum(axis = 1).astype('int')

In [253]:
print(an_df)
print(cn_df)

admission_id
1023    54
1028    54
1029    53
1047    47
1057    53
        ..
9951    54
9955    52
9959    63
9968    53
9972    53
Length: 850, dtype: int32
admission_id
1023    0
1028    1
1029    0
1047    1
1057    1
       ..
9951    0
9955    1
9959    0
9968    0
9972    1
Length: 850, dtype: int32


In [ ]:
#a_df = pd.DataFrame(ad_df["admission_date"].str.split("-", n = 2, expand = False))
#ad_df[['ay','am','ad']] = pd.DataFrame(a_df.admission_date.tolist(), index= a_df.index).astype(int)
#d_df = pd.DataFrame(ad_df["discharge_date"].str.split("/", n = 2, expand = False))
#ad_df[['dd','dm','dy']] = pd.DataFrame(d_df.discharge_date.tolist(), index= d_df.index).astype(int)
#t_df = pd.DataFrame(ad_df["discharge_time"].str.split(":", n = 1, expand = False))
#ad_df[['th','tm']] = pd.DataFrame(t_df.discharge_time.tolist(), index= t_df.index).astype(int)
#ad_df[['ay','dy']] = ad_df[['ay','dy']] - 2000

In [61]:
a_df = admin_df.drop(['patient_id','admission_date','admission_time','discharge_date','discharge_time'], axis = 1)
a_df.set_index('admission_id', inplace = True)
l_df = lab_df.drop(['test_name','result_unit','result_date','result_time'], axis = 1)
l_df.set_index('admission_id', inplace = True)
t_df = tran_df.drop(['issue_date','issue_time'],axis = 1)
t_df.set_index('admission_id', inplace = True)
all_df = a_df.join([l_df,t_df], how = 'left', sort = True)

In [74]:
all_df.describe(include = 'all')

,hospital,age,sex,charlson_comorbidity_index,lap_score,test_code,result_value,rbc_transfusion,platelet_transfusion,plasma_transfusion
count,5231,4604.00000,5231,4970,5231.000000,3609,3609.000000,657,657,657
unique,5,NaN,2,3,NaN,9,NaN,2,2,2
top,St. Michael's Hospital,NaN,F,1,NaN,ALB,NaN,True,False,False
freq,1413,NaN,2832,1818,NaN,1463,NaN,376,653,380
mean,NaN,54.94874,NaN,NaN,27.528197,NaN,81.539762,NaN,NaN,NaN
std,NaN,5.05146,NaN,NaN,21.423248,NaN,94.470176,NaN,NaN,NaN
min,NaN,38.00000,NaN,NaN,0.000000,NaN,0.000000,NaN,NaN,NaN
25%,NaN,51.00000,NaN,NaN,11.000000,NaN,3.600000,NaN,NaN,NaN
50%,NaN,55.00000,NaN,NaN,23.000000,NaN,48.600000,NaN,NaN,NaN
75%,NaN,58.00000,NaN,NaN,40.000000,NaN,140.900000,NaN,NaN,NaN


In [ ]:
### EDA on each dataset

In [ ]:
pd.describe, some graphs, unique values, value_counts()

In [ ]:
### Fix any missing values/ make decisions 

In [ ]:
### Deal with any weirdness in data 
 # - things that may happen when information is manually entered
 # - look at unique strings in columns -> weird spellings?
 # - are things that are supposed to be numerical actually numbers?
 # - are there any clear outliers where maybe something went wrong?

# Show these results

Categorical Column ''''blah blab blah''' is changed to xyz

### Step 2:
Impute the missing age and charlson_comorbidity_index values in any way you see fit. Briefly explain how the imputation method(s) might affect a statistical model that uses these variables as predictors.

### Step 3:
Determine if there is a significant difference in age between patients who had an rbc_transfusion and patients that did not. Determine if there is a significant difference in sex between these groups.

### Step 4:
Fit a linear regression model using the result_value of the “Platelet Count” lab tests as the dependent variable and age, sex, and hospital as the independent variables. Interpret the results.

### Step 5:
Create a plot that demonstrates the relationship between length_of_stay (discharge date and time minus admission date and time) and charlson_comorbidity_index. Briefly describe how you would change the visualization to add age as a third variable.

### Step 6:
Fit a statistical model of your choosing to predict the result_value of the “Hemoglobin” lab tests and evaluate its performance. Explain your choices and how you might improve the model if you had to iterate on it. What other clinical variables would you be interested in adding to the model (they don’t have to be in the provided datasets)?